Устанавливаем необходимые библиотеки

In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip install gTTS
!pip install translate
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215251 sha256=0f25ec899354132269714528eb5ca892354c16b94aceb29c28db8ffd2191b38b
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pyTelegramBotAPI


Реализуем функционал бота

In [ ]:
# импортируем необходимые библиотеки
import telebot
import os
import speech_recognition
from pydub import AudioSegment
from translate import Translator
from gtts import gTTS
from telebot import types
# задаём токен созданного бота в Telegram
bot = telebot.TeleBot('6701808341:AAEBGPCWTB3DiHZo3P2gjgB1VJ-VzZn15kI')

# Создаём "словарь", хранящий язык перевода для каждого пользователя
user_language = {}

# Создаём функцию, начинающую диалог с пользователем
@bot.message_handler(commands=['start'])
def start(message):
    # Устанавливаем язык перевода как None
    user_language[message.chat.id] = None
    # Вызываем функцию, запрашивающую у пользователя язык перевода
    ask_for_translation(message)

# Создаём функцию, выводящую подсказку
@bot.message_handler(commands=['help'])
def help(message):
    bot.reply_to(message, "Я - бот, конвертирующий текстовые сообщения в голосовые с переводом на другой язык. Я могу обратотать текст, написанный на английском языке. Отправьте мне текст, который хотите конвертировать и перевести.")

# Создаём функцию, запрашивающую у пользователя язык перевода
def ask_for_translation(message):
    # Создаём клавиатуру с возможностью выбора только одного варианта
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    item_ru = types.KeyboardButton('Русский')
    item_sr = types.KeyboardButton('Сербский')
    item_zh = types.KeyboardButton('Китайский')
    item_de = types.KeyboardButton('Немецкий')
    item_pl = types.KeyboardButton('Польский')
    item_ja = types.KeyboardButton('Японский')
    item_fr = types.KeyboardButton('Французский')
    item_es = types.KeyboardButton('Испанский')
    item_it = types.KeyboardButton('Итальянский')
    item_el = types.KeyboardButton('Греческий')
    item_ko = types.KeyboardButton('Корейский')
    item_bs = types.KeyboardButton('Боснийский')
    markup.add(item_ru, item_sr, item_zh, item_de, item_pl, item_ja, item_fr, item_es, item_it, item_el, item_ko, item_bs)
    # Отправляем сообщение с запросом языка перевода и клавиатурой
    bot.send_message(message.chat.id, 'На какой язык Вы хотите перевести текст?', reply_markup=markup)
    # Регистрируем следующий шаг, т.е. получение выбранного пользователем языка перевода
    bot.register_next_step_handler(message, get_user_language)

# Создаём функцию для получения выбранного пользователем языка перевода
def get_user_language(message):
    # Задаём условия, если пользователь выбрал тот или иной язык
    if message.text.lower() == 'русский':
        user_language[message.chat.id] = 'ru'
    elif message.text.lower() == 'сербский':
        user_language[message.chat.id] = 'sr'
    elif message.text.lower() == 'китайский':
        user_language[message.chat.id] = 'zh'
    elif message.text.lower() == 'немецкий':
        user_language[message.chat.id] = 'de'
    elif message.text.lower() == 'польский':
        user_language[message.chat.id] = 'pl'
    elif message.text.lower() == 'японский':
        user_language[message.chat.id] = 'ja'
    elif message.text.lower() == 'французский':
        user_language[message.chat.id] = 'fr'
    elif message.text.lower() == 'испанский':
        user_language[message.chat.id] = 'es'
    elif message.text.lower() == 'итальянский':
        user_language[message.chat.id] = 'it'
    elif message.text.lower() == 'греческий':
        user_language[message.chat.id] = 'el'
    elif message.text.lower() == 'корейский':
        user_language[message.chat.id] = 'ko'
    elif message.text.lower() == 'боснийский':
        user_language[message.chat.id] = 'bs'
    # Если пользователь ввёл некорректное значение, то выводим сообщение
    else:
        bot.send_message(message.chat.id, 'Пожалуйста, выберите язык из представленного списка.')
        # Вызываем функцию, запрашивающую у пользователя язык перевода
        ask_for_translation(message)

    # Отправляем сообщение с просьбой отправить текстовое сообщение
    bot.send_message(message.chat.id, 'Пожалуйста, отправьте текстовое сообщение.')

@bot.message_handler(content_types=['text'])
def translate_text_to_voice(message):
  # Если язык перевода не был установлен, то вызываем функцию, запрашивающую у пользователя язык перевода
  if user_language.get(message.chat.id) is None:
        ask_for_translation(message)
  else:
        text = message.text
        # Создаём объект переводчика на выбранный пользователем язык перевода
        translator = Translator(to_lang=user_language[message.chat.id])
        # Переводим текст на выбранный пользователем язык
        translation = translator.translate(text)
        # Создаём объект синтезатора речи на выбранном пользователем языке
        tts = gTTS(text=translation, lang=user_language[message.chat.id])
        # Сохраняем синтезированный голос в файл
        tts.save("voice.mp3")
        # Открываем файл с синтезированным голосом для чтения в двоичном режиме
        audio = open("voice.mp3", "rb")
        # Отправляем голосовое сообщение с синтезированным голосом
        bot.send_voice(message.chat.id, audio)
        # Удаляем временный файл с синтезированным голосом
        os.remove("voice.mp3")
# запускаем бота
bot.polling()